<hr>
<div style="background-color: lightgray; padding: 20px; color: black;">
<div>
<img src="https://th.bing.com/th/id/R.3cd1c8dc996c5616cf6e65e20b6bf586?rik=09aaLyk4hfbBiQ&riu=http%3a%2f%2fcidics.uanl.mx%2fwp-content%2fuploads%2f2016%2f09%2fcimat.png&ehk=%2b0brgMUkA2BND22ixwLZheQrrOoYLO3o5cMRqsBOrlY%3d&risl=&pid=ImgRaw&r=0" style="float: right; margin-right: 30px;" width="200"/> 
<font size="6.5" color="8C3061"><b>Rest-Mex: Research on Sentiment Analysis Task for Mexican Tourist Texts 🇲🇽</b></font> <br>
<font size="5.5" color="8C3061"><b>Análisis de Datos</b></font> 
</div>
<div style="text-align: left">  <br>
Edison David Serrano Cárdenas. <br>
MSc en Matemáticas Aplicadas <br>
CIMAT - Sede Guanajuato <br>
</div>

</div>
<hr>

*Cargar librerias:*

### <font color="8C3061" >**Gráficas de la trazas:**</font> 

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import corner
import seaborn as sns
sns.set_theme(style="whitegrid")
np.random.seed(2025)


from forward_mapping import Forward_mapping